<a href="https://colab.research.google.com/github/13194307/UTS_ML2019_ID13194307/blob/master/ML_A2/NeuralNetwork-backup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO LIST:


*   Finish off backpropagation (currently only updates output layer weights)
*   Perhaps add different kinds of layer classes (one for Dense layers, one for output)



In [0]:
import math
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from scipy.special import softmax
from scipy import stats

In [0]:
class NeuralNetwork:
    class Layer:
        class Neuron:
            def __init__(self, inputShape):
                self.weights, self.bias = self.initialiseWeights(inputShape)
                
            def initialiseWeights(self, inputShape):
                weights = np.array([np.random.randn() for _ in range(0, inputShape)]) / math.sqrt(inputShape)
                bias = np.random.randn()
                
                return weights, bias
            
            def getWeights(self):
                return self.weights
            
            def generateNeuronOutput(self, x):
                return np.dot(self.weights, x) + self.bias
            
            def updateWeights(self, update):
                print("Before:", self.weights)
                self.weights -= update
                print("After:", self.weights)
            
        
        def __init__(self, layerType, neuronsPerLayer, inputShape):
            self.layerType = layerType
            self.numNeurons = neuronsPerLayer
            self.inputShape = inputShape
            self.neurons = [self.Neuron(inputShape) for _ in range(neuronsPerLayer)]
            self.dWeightedSum_dWeights = []
            self.dWeightedSum_dInput = []
            
            if layerType == "Output":
                self.dSoftmax = []
            elif layerType == "Dense":
                self.dRelu = []
            
        def getInputShape(self):
            return self.inputShape
        
        def getNumNeurons(self):
            return self.numNeurons
        
        def getDSoftmax(self):
            temp = self.dSoftmax
            self.dSoftmax = []
            return temp
        
        def getDWeightedSum_dWeights(self):
            temp = self.dWeightedSum_dWeights
            self.dWeightedSum_dWeights = []
            return temp
        
        def getDWeightedSum_dInput(self):
            temp = self.dWeightedSum_dInput
            self.dWeightedSum_dInput = []
            return temp
        
        def getDRelu(self):
            temp = self.dRelu
            self.dRelu = []
            return temp
        
        def generateLayerOutput(self, x):
            layerOutput = np.array([])
            change = [self.calcDWeightedSum_dWeights(x) for _ in range(self.numNeurons)]
            self.dWeightedSum_dWeights.append(change)
            
            change = []
            
            for neuron in self.neurons:
                change.append(neuron.getWeights())
                neuronOutput = neuron.generateNeuronOutput(x)
                layerOutput = np.append(layerOutput, neuronOutput)
                
            self.dWeightedSum_dInput.append(change)
               
            print(self.layerType, ":", layerOutput)
            if self.layerType == "Dense":
                #Leaky ReLU activation
                change = self.calcDRelu(layerOutput)
                self.dRelu.append(change)
                layerOutput[layerOutput < 0] *= 0.01
            elif self.layerType == "Output":
                if len(layerOutput) == 1:
                    #Sigmoid activation
                    layerOutput = 1 / (1 + math.exp(-1*layerOutput[0]))
                    #print("Probabilities: ", layerOutput)
                else:
                    #Softmax activation
                    layerOutput = np.exp(layerOutput)/sum(np.exp(layerOutput))
                    change = self.calcDSoftmax(layerOutput)
                    self.dSoftmax.append(change)
                    #print(self.dSoftmax)
                    #print("Probabilities: ", layerOutput)
            else:
                raise NotImplementedError
                    
            print("After activation - ", layerOutput)
            return layerOutput
        
        # Derivative of softmax with respect to weighted sum/dot product
        def calcDSoftmax(self, prob):
            n = len(prob)
            return [prob[i]-(prob[i]**2) for i in range(n)]
        
        # Derivative of weighted sum with respect to the weights
        # This function is kinda pointless as it just returns the argument
        # passed to it unaltered but I added it in to remind me that this
        # is the derivative.
        def calcDWeightedSum_dWeights(self, x):
            return x
        
        # Derivative of weighted sum with respect to the input provided
        # Also a redundant function, and was also added in for the same
        # reason as above
        def calcDWeightedSum_dInput(self, weights):
            return weights
        
        # Derivative of Leaky ReLU with respect to weighted sum/dot product
        def calcDRelu(self, input_vector):
            derivative = np.array(input_vector)
            derivative[derivative > 0] = 1
            derivative[derivative <= 0] = 0.01
            return derivative
        
        def updateWeights(self, update):
            for i in range(self.numNeurons):
                self.neurons[i].updateWeights(update[i])
            
        def __str__(self):
            output = ""
            for neuron in self.neurons:
                weights, bias = neuron.getWeightsAndBias()
                output+="\t"
                
                for j in range(0, len(weights)):
                    output+=("w{}: {}, ".format(j, weights[j]))
                    
                output+=("b0: {}\n".format(bias))
            
            return output
        
        
        
        
        
    
    def __init__(self):
        self.layers = []
        self.numLayers = 0
        
    def addLayer(self, layerType, neuronsPerLayer, inputShape=None):
        if inputShape == None:
            inputShape = self.layers[-1].getNumNeurons()
            
        self.layers.append(self.Layer(layerType, neuronsPerLayer, inputShape))
        self.numLayers+=1
        
    def predict(self, x, labels, batch_size=2, step_size = 0.1):
        probabilities = []
        dError = []
        counter = 0
        
        for i in range(len(x)):
            prob = self.feedForward(x[i])
            change = self.calcDError(prob, labels[i])
            dError.append(change)
            counter+=1
            
            if counter == batch_size:
                counter = 0
                self.backPropagation(dError, step_size, batch_size)
                dError = []
                
            #print(pred)
            probabilities.append(prob)
      
        predictions = np.argmax(probabilities, axis=1)
        #loss = self.calcTotalLoss(probabilities, labels)
        #print(loss)
        return predictions
    
    def feedForward(self, x):
        lastLayerOutput = x
        
        for layer in self.layers:
            lastLayerOutput = layer.generateLayerOutput(lastLayerOutput)
            
        return lastLayerOutput
    
    def backPropagation(self, dError, step_size, batch_size=None):
        #Only updates output layer weights for now
        d1 = np.array(dError)
        d2 = np.array(self.layers[-1].getDSoftmax())
        d3 = np.array(self.layers[-1].getDWeightedSum_dWeights())
        shape = np.shape(d3[0])
        print(shape)
        onesArray = np.ones((shape[1], shape[0]))
        d1 = [np.transpose(d1[i]*onesArray) for i in range(batch_size)]
        d2 = [np.transpose(d2[i]*onesArray) for i in range(batch_size)]
        print("d1:", d1)
        print("d2:", d2)
        print("d3:", d3[0])
        changeOutputWeights = [d1[i]*d2[i]*d3[i] for i in range(len(d3))]
        avgChange = (sum(changeOutputWeights)/batch_size) * step_size
        #print("change:", changeOutputWeights)
        #print("fin change:", avgChange)
        self.layers[-1].updateWeights(avgChange)
        
        # Derivatives for H[-1] 
        d4 = np.array(self.layers[-1].getDWeightedSum_dInput())
        d5 = np.array(self.layers[-2].getDRelu())
        d6 = np.array(self.layers[-2].getDWeightedSum_dWeights())
        print("d4:", d4)
        print("d6:", d6)
        dTotalError = np.array([np.sum(d1[i]*d2[i]*d4[i], axis=1) for i in range(batch_size)])
        #numWeights = len(dTotalError[0, 0])
        print("dTotal before:",dTotalError)
        numNeurons = len(dTotalError[0])
        shape = np.shape(d6[0])
        print(shape)
        onesArray = np.ones((shape[1], shape[0]))
        print(onesArray)
        d5 =[np.transpose(d5[i]*onesArray) for i in range(batch_size)]
        print("d5:", d5)
        #dTotalError = [np.transpose(dTotalError[i]*onesArray) for i in range(batch_size)]
        dTotalError = [[[dTotalError[j, i] for _ in range(shape[1])] for i in range(len(dTotalError[0]))] for j in range(batch_size)]
        print("dTotalError sum:", dTotalError)
        changeHiddenWeights = [d5[i]*d6[i]*dTotalError[i] for i in range(batch_size)]
        avgChange = (sum(changeHiddenWeights)/batch_size) * step_size
        
        
        print("Change:", changeHiddenWeights)
        print("Avg:", avgChange)
        
        self.layers[-2].updateWeights(avgChange)
        # End of derivatives for H[-1] 
        
        # Start of loop experiments
        #cumulativeDTotalError = [[d2[i] for _ in range()]]
        
        #for i in range(-2, -1*(self.numLayers+1), -1):
            #d4 = np.array(self.layers[-1].getDWeightedSum_dInput())
            
        # End of loop experiments
    
    #Derivative of error (cross-entropy) with respect to softmax probabilities
    def calcDError(self, prob, actual):
        n = len(actual)
        return [-1*(actual[i]/prob[i]) + ((1 - actual[i])*(-1/(1-prob[i]))) for i in range(n)]
    
    def calcTotalLoss(self, prob, actual):
        #REDO THIS TO WORK WITH ONE HOT ENCODING
        n = len(actual)
        loss = -1*sum(np.log([prob[i][actual[i]] for i in range(n)]))/n
        return loss
        
    def __str__(self):
        output = ""
        
        for i in range(0, self.numLayers):
            output+=("Layer {}:\n".format(i+1))
            output+=str(self.layers[i])
        
        return output

In [0]:
nn = NeuralNetwork()
nn.addLayer("Dense", 3, inputShape=4)
nn.addLayer("Dense", 4)
nn.addLayer("Dense", 4)
nn.addLayer("Output", 3)
#print(nn)

In [0]:
from sklearn.datasets import load_iris

iris_X, iris_y = load_iris(True)

In [0]:
from sklearn.preprocessing import MinMaxScaler

iris_X_trimmed = iris_X

scaler = MinMaxScaler()
iris_X_scaled = scaler.fit_transform(iris_X_trimmed)
iris_X_zscore = stats.zscore(iris_X_trimmed)

In [0]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
labels = lb.fit_transform(iris_y)

In [201]:
pred = nn.predict(iris_X_zscore[0:2, :], labels[0:2])
accuracy_score(pred, iris_y[0:2])

Dense : [0.64372026 1.19023898 2.26524772]
After activation -  [0.64372026 1.19023898 2.26524772]
Dense : [-1.00577828  1.50986484  1.30950984  3.4854111 ]
After activation -  [-0.01005778  1.50986484  1.30950984  3.4854111 ]
Dense : [ 0.62465532 -1.97143923  2.58909454  2.20948385]
After activation -  [ 0.62465532 -0.01971439  2.58909454  2.20948385]
Output : [-0.53929152  0.52502287 -3.56404311]
After activation -  [0.25332998 0.73436582 0.01230421]
Dense : [-0.96743041  1.12217199  1.14611975]
After activation -  [-0.0096743   1.12217199  1.14611975]
Dense : [-0.98324391  1.85733489  0.5500001   1.88902051]
After activation -  [-0.00983244  1.85733489  0.5500001   1.88902051]
Dense : [ 0.28423615 -1.66167811  2.87364482  1.88596996]
After activation -  [ 0.28423615 -0.01661678  2.87364482  1.88596996]
Output : [-1.07794482  0.13188728 -3.37382443]
After activation -  [0.22453773 0.75285742 0.02260484]
(3, 4)
d1: [array([[-3.94742072, -3.94742072, -3.94742072, -3.94742072],
       [-

ValueError: ignored

In [0]:
print(iris_y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [198]:
x = [[1, 2, 3], [4, 5, 6]]

y = [[[x[j][i] for _ in range(4)] for i in range(len(x[0]))] for j in range(len(x))]
y

[[[1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3, 3]],
 [[4, 4, 4, 4], [5, 5, 5, 5], [6, 6, 6, 6]]]